In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
import os

# 크롬 옵션 설정
options = Options()
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

csv_filename = "vet6390_blog_posts_images_links.csv"

# CSV 파일 준비
file_exists = os.path.exists(csv_filename)
f = open(csv_filename, 'a', newline='', encoding='utf-8-sig')
writer = csv.writer(f)
if not file_exists:
    writer.writerow(["이미지 URL", "글 주소"])

# URL 접속
url = "https://m.blog.naver.com/PostList.naver?blogId=vet6390&categoryName=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%9D%B4%EC%95%BC%EA%B8%B0&categoryNo=20&listStyle=post&logCode=0&tab=1"
driver.get(url)
time.sleep(3)

scroll_count = 0

while True:
    try:
        list_block = driver.find_element(By.CLASS_NAME, "list_block__XlpUJ")
        post_items = list_block.find_elements(By.CLASS_NAME, "item__PxpH8")
    except Exception as e:
        print(f"리스트 찾기 실패: {e}")
        break

    print(f"현재 수집 가능한 글 수: {len(post_items)}개 (스크롤 {scroll_count}회)")

    if not post_items:
        print("글 항목을 찾을 수 없습니다. 종료합니다.")
        break

    found_new = False

    for i in range(len(post_items)):
        try:
            list_block = driver.find_element(By.CLASS_NAME, "list_block__XlpUJ")
            post_items = list_block.find_elements(By.CLASS_NAME, "item__PxpH8")

            driver.execute_script("arguments[0].scrollIntoView(true);", post_items[i])
            time.sleep(1)

            link = post_items[i].find_element(By.TAG_NAME, "a")
            href = link.get_attribute('href')
            if not href:
                print("href가 없습니다. 스킵합니다.")
                continue

            driver.get(href)
            print(f"{i + 1}번째 글 수집 중...")

            wait = WebDriverWait(driver, 10)
            image_url = "이미지 없음"

            # 대표 이미지 찾기 시도
            try:
                image_element = wait.until(EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.se-main-container img")
                ))
                image_url = image_element.get_attribute("src")
            except:
                try:
                    image_element = driver.find_element(By.CSS_SELECTOR, "img")
                    image_url = image_element.get_attribute("src")
                except:
                    print("이미지 없음")

            post_url = driver.current_url
            print(f"→ 이미지 URL: {image_url}")
            print(f"→ 글 주소: {post_url}")

            writer.writerow([image_url, post_url])
            f.flush()  # 실시간 저장
            found_new = True
            print("수집 완료\n")

            driver.back()
            time.sleep(2)

        except Exception as e:
            print(f"에러 발생: {e}")
            driver.get(url)
            time.sleep(3)
            break

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_count += 1
    time.sleep(2)

    if not found_new:
        print("새 글 없음, 크롤링 종료.")
        break

f.close()
print(f"\nCSV 저장 완료! 파일명: {csv_filename}")

driver.quit()


현재 수집 가능한 글 수: 24개 (스크롤 0회)
1번째 글 수집 중...
→ 이미지 URL: https://mblogthumb-phinf.pstatic.net/MjAyMzA5MDlfMjM4/MDAxNjk0MjU2NjkxMzY0.tMfLbdp0QWelWRx6zJrmEAEz4cQ8xkFntQ3ouaP65wog.lt6be9RASGugnZsFQPoqDiQ8VvpedRteZ2mCwR2SwKEg.PNG.vet6390/%EA%B0%95%EB%82%A8_%EA%B0%95%EC%95%84%EC%A7%80_%ED%94%BC%EB%B6%80%EB%B3%91_%EC%B9%98%EB%A3%8C.PNG?type=w800
→ 글 주소: https://m.blog.naver.com/vet6390/223206908835?referrerCode=1
수집 완료

2번째 글 수집 중...
→ 이미지 URL: https://mblogthumb-phinf.pstatic.net/MjAyMzA5MDdfMTMw/MDAxNjk0MDQ2MjU0NDk1.b9HKSEX-U0s27gVk0y6e1gcLn5mHmsZJVEEMuSN8GB8g.bVHIOCesHScJK6ye9i52dz_6dhHKoqU_0tAn6bJRCf8g.PNG.vet6390/%EA%B0%95%EB%82%A8_%EA%B0%95%EC%95%84%EC%A7%80_%EC%8B%9D%EC%A4%91%EB%8F%85_%EC%B9%98%EB%A3%8C_%EB%8F%99%EB%AC%BC%EB%B3%91%EC%9B%90.PNG?type=w800
→ 글 주소: https://m.blog.naver.com/vet6390/223204684520?referrerCode=1
수집 완료

3번째 글 수집 중...
→ 이미지 URL: https://mblogthumb-phinf.pstatic.net/MjAyMzA5MDRfMTc3/MDAxNjkzODE5MzM0NDY5.1Pw67wtpt_m6_dUJP2BGSjHYqYy5fooIz3iv8cMLeucg.arpJuzBzOSEiGxSZAG

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
import os

# 크롬 옵션 설정
options = Options()
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 기존 URL+이미지 파일
input_csv = "post_links_images_renamed.csv"
output_csv = "post_links_full.csv"

# 기존 데이터 불러오기
with open(input_csv, 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

# 새 CSV 파일 열기
with open(output_csv, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(["title", "content", "URL", "image"])

    for idx, row in enumerate(rows, 1):
        url = row["URL"]
        image_url = row["image"]

        try:
            driver.get(url)
            print(f"{idx}번째 글 처리 중...")

            wait = WebDriverWait(driver, 10)

            # 제목
            try:
                title_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.se-module.se-module-text.se-title-text")))
                title = title_element.text.strip()
            except:
                title = "제목 없음"

            # 본문
            try:
                content_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "se-main-container")))
                content = content_element.text.strip()
            except:
                content = "본문 없음"

            writer.writerow([title, content, url, image_url])
            f.flush()
            print("→ 수집 완료\n")

        except Exception as e:
            print(f"❌ 에러 발생 ({url}): {e}")
            continue

driver.quit()
print(f"\n✅ 모든 수집 완료! 저장 파일: {output_csv}")


1번째 글 처리 중...
→ 수집 완료

2번째 글 처리 중...
→ 수집 완료

3번째 글 처리 중...
→ 수집 완료

4번째 글 처리 중...
→ 수집 완료

5번째 글 처리 중...
→ 수집 완료

6번째 글 처리 중...
→ 수집 완료

7번째 글 처리 중...
→ 수집 완료

8번째 글 처리 중...
→ 수집 완료

9번째 글 처리 중...
→ 수집 완료

10번째 글 처리 중...
→ 수집 완료

11번째 글 처리 중...
→ 수집 완료

12번째 글 처리 중...
→ 수집 완료

13번째 글 처리 중...
→ 수집 완료

14번째 글 처리 중...
→ 수집 완료

15번째 글 처리 중...
→ 수집 완료

16번째 글 처리 중...
→ 수집 완료

17번째 글 처리 중...
→ 수집 완료

18번째 글 처리 중...
→ 수집 완료

19번째 글 처리 중...
→ 수집 완료

20번째 글 처리 중...
→ 수집 완료

21번째 글 처리 중...
→ 수집 완료

22번째 글 처리 중...
→ 수집 완료

23번째 글 처리 중...
→ 수집 완료

24번째 글 처리 중...
→ 수집 완료

25번째 글 처리 중...
→ 수집 완료

26번째 글 처리 중...
→ 수집 완료

27번째 글 처리 중...
→ 수집 완료

28번째 글 처리 중...
→ 수집 완료

29번째 글 처리 중...
→ 수집 완료

30번째 글 처리 중...
→ 수집 완료

31번째 글 처리 중...
→ 수집 완료

32번째 글 처리 중...
→ 수집 완료

33번째 글 처리 중...
→ 수집 완료

34번째 글 처리 중...
→ 수집 완료

35번째 글 처리 중...
→ 수집 완료

36번째 글 처리 중...
→ 수집 완료

37번째 글 처리 중...
→ 수집 완료

38번째 글 처리 중...
→ 수집 완료

39번째 글 처리 중...
→ 수집 완료

40번째 글 처리 중...
→ 수집 완료

41번째 글 처리 중...
→ 수집 완료

42번째 글 처리 중...
→ 수집 완료

4